In [2]:
import nltk

In [13]:
import re
import csv

import nltk.classify


def replaceTwoOrMore(s):
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL)
    return pattern.sub(r"\1\1", s)


def processTweet(tweet):
    tweet = tweet.lower()
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', tweet)
    tweet = re.sub('@[^\s]+', 'AT_USER', tweet)
    tweet = re.sub('[\s]+', ' ', tweet)
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    tweet = tweet.strip('\'"')
    return tweet


def getStopWordList(stopWordListFileName):
    stopWords = []
    stopWords.append('AT_USER')
    stopWords.append('URL')

    fp = open(stopWordListFileName, 'r')
    line = fp.readline()
    while line:
        word = line.strip()
        stopWords.append(word)
        line = fp.readline()
    fp.close()
    return stopWords


def getFeatureVector(tweet, stopWords):
    featureVector = []
    words = tweet.split()
    for w in words:

        w = replaceTwoOrMore(w)

        w = w.strip('\'"?,.')

        val = re.search(r"^[a-zA-Z][a-zA-Z0-9]*[a-zA-Z]+[a-zA-Z0-9]*$", w)

        if (w in stopWords or val is None):
            continue
        else:
            featureVector.append(w.lower())
    return featureVector


def extract_features(tweet):
    tweet_words = set(tweet)
    features = {}
    for word in featureList:
        features['contains(%s)' % word] = (word in tweet_words)
    return features


inpTweets = csv.reader(open('data/training.csv', 'r'), delimiter=',', quotechar='|')
stopWords = getStopWordList('data/stopwordsID.txt')
count = 0;
featureList = []
tweets = []
for row in inpTweets:
    sentiment = row[0]
    tweet = row[1]
    processedTweet = processTweet(tweet)  # preprocessing
    featureVector = getFeatureVector(processedTweet, stopWords)  # get feature vector
    featureList.extend(featureVector)
    tweets.append((featureVector, sentiment));

featureList = list(set(featureList))
training_set = nltk.classify.util.apply_features(extract_features, tweets)
NBClassifier = nltk.NaiveBayesClassifier.train(training_set)

testTweet = 'pantai di lombok bersih bersih. pasirnya juga indah'
processedTestTweet = processTweet(testTweet)
sentiment = NBClassifier.classify(extract_features(getFeatureVector(processedTestTweet, stopWords)))

0.85

In [12]:
print("Test Tweets = %s, Sentiment = %s\n" % (testTweet, sentiment))
# print("Show Most Informative Features", NBClassifier.show_most_informative_features(32))
# print()
# print("Extract Features", extract_features(testTweet.split()))

print("Akurasi Hasil Klasifikasi :", (nltk.classify.accuracy(NBClassifier, training_set)) * 100)
# print("Akurasi Hasil Klasifikasi :", accuracy_score(testTweet, sentiment))

# kal = getFeatureVector(processTweet(testTweet), stopWords)
# kal = " ".join(str(x) for x in kal)
# print(kal)
# d = {}
# for word in kal.split():
#     word = int(word) if word.isdigit() else word
#     if word in d:
#         d[word] += 1
#     else:
#         d[word] = 1

# print(d)

Test Tweets = pantai di lombok bersih bersih. pasirnya juga indah, Sentiment = positive

Akurasi Hasil Klasifikasi : 85.0
